# CopyCat: Example Usage on the Web Track Runs

- Use cases of CopyCat:
  - Deduplicate complete corpora on Spark/Hadoop
  - Deduplicate run/qrel files on your laptop

In [1]:
# Show Usage of copy cat.
!../bash/copy-cat.sh -h

Error: Could not find or load main class de.webis.cikm20_duplicates.app.DeduplicateTrecRunFile
Caused by: java.lang.ClassNotFoundException: de.webis.cikm20_duplicates.app.DeduplicateTrecRunFile


## SERP Deduplication

- Situation:
  - Near-Duplicates are removed from the Index
    - With SimHash (3+5-grams) + Canonical-URL

- Is it still necessary to remove near-duplicates from the SERP?


## WIP: SERP Deduplication for a single run

In [ ]:
# Deduplicate a single run-file (please remove the output file to run copy-cat from scratch)

!../bash/copy-cat.sh \
    --output run-UAMSA10d2a8-deduplication.jsonl \
    --input input.UAMSA10d2a8 \
    --similarities "url s3 cosine(3+5-grams) cosine(8-grams) cosine(1-grams) simhash(1-gramms) simhash(3+5-gramms)" \
    --s3Threshold 0.7 \
    --threads 5 \
    --documents ChatNoirMapfiles

In [2]:
import json
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from trectools import TrecQrel

qrels = TrecQrel('qrels.web.101-150.txt')

def eval_with_threshold(threshold, run_file_name):
    rows = []
    print("Process " + run_file_name)
    with open(run_file_name) as jsonl_file:
        for jsonl in jsonl_file:
            dedup_data = json.loads(jsonl)
            topic = dedup_data['topic']
            judged_docs = qrels.get_document_names_for_topic(int(topic))
            
            for sim in tqdm(dedup_data['similarities']):
                is_judged = sim['firstId'] in judged_docs or sim['secondId'] in judged_docs
                is_relevant = False
                is_irrelevant = False
                
                if is_judged:
                    judgment_a = qrels.get_judgement(sim['firstId'], int(topic))
                    judgment_b = qrels.get_judgement(sim['secondId'], int(topic))
                    is_irrelevant = judgment_a == 0 or judgment_b == 0
                    is_relevant = not is_irrelevant
                
                rows += [{
                    'topic': topic,
                    'judged': is_judged,
                    'relevant': is_relevant,
                    'irrelevant': is_irrelevant,
                    'near-duplicate': sim['similarities']['s3'] >=  threshold,
                    'simhash(3+5-grams)': sim['similarities']['simhash(3+5-grams)'] > 0.94,
                    'simhash(1-grams)': sim['similarities']['simhash(1-grams)'] > 0.94,
                    'url': sim['similarities']['url'] > 0.5,
                    'copy-cat': ((sim['similarities']['simhash(3+5-grams)'] > 0.94) or ((sim['similarities']['url'] > 0.5) and (sim['similarities']['simhash(1-grams)'] > 0.94)))
                }]

    return rows

def eval_runs_with_threshold(threshold, run_files):
    rows = []
    for r in run_files:
        rows += eval_with_threshold(threshold, r)
    
    return pd.DataFrame(rows)

In [ ]:
%time
DEDUP_TARGET_DIR='../../../top-100-deduplication/'
ALL_DIRS=!ls $DEDUP_TARGET_DIR
ALL_DIRS=!ls $DEDUP_TARGET_DIR
ALL_DIRS = [DEDUP_TARGET_DIR + i for i in ALL_DIRS]


df = eval_runs_with_threshold(0.82, ALL_DIRS)

CPU times: user 6 µs, sys: 6 µs, total: 12 µs
Wall time: 21.2 µs


  0%|          | 0/148 [00:00<?, ?it/s]

Process ../../../top-100-deduplication/input.2011SiftR1.jsonl


 18%|█▊        | 28/152 [00:00<00:00, 273.79it/s]

Process ../../../top-100-deduplication/input.2011SiftR2.jsonl


 87%|████████▋ | 84/97 [00:00<00:00, 828.88it/s]

Process ../../../top-100-deduplication/input.2011SiftR3.jsonl


  0%|          | 0/34 [00:00<?, ?it/s]

Process ../../../top-100-deduplication/input.DFalah11.jsonl


100%|██████████| 13/13 [00:00<00:00, 187.82it/s]
0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00, 299.01it/s]
0it [00:00, ?it/s]
100%|██████████| 11/11 [00:00<00:00, 186.17it/s]

In [62]:
, 

def precision_score(df, approach):
    from sklearn.metrics import precision_score
    return "{:0.3f}".format(precision_score(y_true=df['near-duplicate'], y_pred=df[approach]))

def recall_score(df, approach):
    from sklearn.metrics import recall_score
    return "{:0.3f}".format(recall_score(y_true=df['near-duplicate'], y_pred=df[approach]))


print('# top 100')
print('CopyCat (Precision):', precision_score(df, 'copy-cat'))
print('CopyCat (Recall):', recall_score(df, 'copy-cat'))


print('SimHash(3+5-grams) (Precision):', precision_score(df, 'simhash(3+5-grams)'))
print('SimHash(3+5-grams) (Recall):', recall_score(df, 'simhash(3+5-grams)'))

print('SimHash(1-grams) (Precision):', precision_score(df, 'simhash(1-grams)'))
print('SimHash(1-grams) (Recall):', recall_score(df, 'simhash(1-grams)'))


print('\n\n# Judged@top 100')
print('Copy-Cat (Precision):', precision_score(df[df['judged']], 'copy-cat'))
print('Copy-Cat (Recall):', recall_score(df[df['judged']], 'copy-cat'))

print('SimHash(3+5-grams) (Precision):', precision_score(df[df['judged']], 'simhash(3+5-grams)'))
print('SimHash(3+5-grams) (Recall):', recall_score(df[df['judged']], 'simhash(3+5-grams)'))

print('SimHash(1-grams) (Precision):', precision_score(df[df['judged']], 'simhash(1-grams)'))
print('SimHash(1-grams) (Recall):', recall_score(df[df['judged']], 'simhash(1-grams)'))


print('\n\n# Relevant@top 100')
print('Copy-Cat (Precision):', precision_score(df[(df['judged']) & (df['relevant'])], 'copy-cat'))
print('Copy-Cat (Recall):', recall_score(df[(df['judged']) & (df['relevant'])], 'copy-cat'))

print('SimHash(3+5-grams) (Precision):', precision_score(df[(df['judged']) & (df['relevant'])], 'simhash(3+5-grams)'))
print('SimHash(3+5-grams) (Recall):', recall_score(df[(df['judged']) & (df['relevant'])], 'simhash(3+5-grams)'))

print('SimHash(1-grams) (Precision):', precision_score(df[(df['judged']) & (df['relevant'])], 'simhash(1-grams)'))
print('SimHash(1-grams) (Recall):', recall_score(df[(df['judged']) & (df['relevant'])], 'simhash(1-grams)'))


# top 100
CopyCat (Precision): 0.999
CopyCat (Recall): 0.733
SimHash(3+5-grams) (Precision): 0.999
SimHash(3+5-grams) (Recall): 0.634
SimHash(1-grams) (Precision): 0.813
SimHash(1-grams) (Recall): 0.918


# Judged@top 100
Copy-Cat (Precision): 0.999
Copy-Cat (Recall): 0.765
SimHash(3+5-grams) (Precision): 0.998
SimHash(3+5-grams) (Recall): 0.650
SimHash(1-grams) (Precision): 0.958
SimHash(1-grams) (Recall): 0.951


# Relevant@top 100
Copy-Cat (Precision): 1.000
Copy-Cat (Recall): 0.742
SimHash(3+5-grams) (Precision): 1.000
SimHash(3+5-grams) (Recall): 0.531
SimHash(1-grams) (Precision): 0.946
SimHash(1-grams) (Recall): 0.988


In [59]:
df[df['judged']]['judged'].unique()

array([ True])

In [36]:
%time
DEDUP_TARGET_DIR='../../../top-100-deduplication/'
ALL_DIRS=!ls $DEDUP_TARGET_DIR
ALL_DIRS=!ls $DEDUP_TARGET_DIR
ALL_DIRS = [DEDUP_TARGET_DIR + i for i in ALL_DIRS]


eval_runs_with_threshold(0.82, ALL_DIRS[0:1]).mean()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
Process ../../../top-100-deduplication/input.2011SiftR1.jsonl
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged: 

Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrel

Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  Fal

Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelev

Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  False ; relevant:  False ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant

...

Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant: 

Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  True ; irrelevant:  False
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:

Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:

Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:  True
Judged:  True ; relevant:  False ; irrelevant:

Recall (SimHash 3+5-grams)       0.664184
Recall (SimHash 1-grams)         0.921189
Recall (url)                     0.324333
Recall (CopyCat)                 0.740088
Precision (SimHash 3+5-grams)    0.997952
Precision (SimHash 1-grams)      0.918107
Precision (Url)                  1.000000
Precision (CopyCat)              0.998264
dtype: float64

In [114]:
eval_with_threshold(0.82).mean()

Recall (SimHash 3+5-grams)       0.688893
Recall (SimHash 1-grams)         0.849880
Precision (SimHash 3+5-grams)    0.985294
Precision (SimHash 1-grams)      0.909336
dtype: float64

In [115]:
eval_with_threshold(0.90).mean()

Recall (SimHash 3+5-grams)       0.807388
Recall (SimHash 1-grams)         0.929377
Precision (SimHash 3+5-grams)    0.941176
Precision (SimHash 1-grams)      0.798057
dtype: float64

## Next Steps:

- Improve efficiency of "local" copy-cat
- Evaluations:
  - Recall:
    - @100, @1000, @Judged, @Relevant, @Irrelevant
  - At which Positions occur duplicates?
    - Duplicates within the top k
- Evaluation Notebooks (all available runs):
  - ClueWeb
  - Robust04
  - Argsme
- Relevance Transfer:
  - Deduplicate on the top-10000 results for each topic with CopyCat
- Finish Anserini-Integration
  - Use jigsaw modules to separate ChatNoir from Anserini